
# Table of Contents

1.  [Classes:](#org29a813e)
    1.  [Galactic:](#org4b273fc)
        1.  [16 (924) Periodic (technically) but in practice some may be transient in our data - we think it is an eclipsing binary](#org55802c3)
        2.  [53 (30) Periodic (very), low time scale range, seems to be about a year? - probably a variable star of some sort - Mira variables probably](#org88b82c9)
        3.  [6 (151) Transient, varying time scales, unknown what it actually is possibly nova of some sort](#orgb4fcfef)
        4.  [65 (981) Periodic (sort of) (At least 2 year time scale, very very short burst) - Is it a flare? I really have no idea](#org676f839)
        5.  [92 (239) Very periodic - Very short time scale (smaller than cadence) - "Some kind of variable star"](#orge9488ee)
    2.  [Extragalactic:](#org42a8b71)
        1.  [15 Transient - Maybe supernova - ramp up? weird.](#org9e8d5fa)
        2.  [42 Transient - Some kind of supernova - not Ia](#orgfbfecf8)
        3.  [52 Transient - idk](#orgdd32283)
        4.  [62 Transient](#org17818a0)
        5.  [64 Transient happens in only one band very short time scale emission line?](#orgd975edb)
        6.  [67 Transient](#org1032894)
        7.  [88 Periodic! finally I think it is an AGN](#org336f1fb)
        8.  [90 Transient](#org0d56f34)
        9.  [95 Transient - Not supernova??](#orgf4cb7b7)


<a id="org29a813e"></a>

## Classes:


<a id="org4b273fc"></a>

### Galactic:


<a id="org55802c3"></a>

 16 (924) Periodic (technically) but in practice some may be transient in our data - we think it is an eclipsing binary


<a id="org88b82c9"></a>

 53 (30) Periodic (very), low time scale range, seems to be about a year? - probably a variable star of some sort - Mira variables probably


<a id="orgb4fcfef"></a>

 6 (151) Transient, varying time scales, unknown what it actually is possibly nova of some sort


<a id="org676f839"></a>

 65 (981) Periodic (sort of) (At least 2 year time scale, very very short burst) - Is it a flare? I really have no idea


<a id="orge9488ee"></a>

 92 (239) Very periodic - Very short time scale (smaller than cadence) - "Some kind of variable star"


<a id="org42a8b71"></a>

### Extragalactic:


<a id="org9e8d5fa"></a>

 15 Transient - Maybe supernova - ramp up? weird.


<a id="orgfbfecf8"></a>

 42 Transient - Some kind of supernova - not Ia


<a id="orgdd32283"></a>

 52 Transient - idk


<a id="org17818a0"></a>

 62 Transient


<a id="orgd975edb"></a>

 64 Transient happens in only one band very short time scale emission line?


<a id="org1032894"></a>

 67 Transient


<a id="org336f1fb"></a>

 88 Periodic! finally I think it is an AGN


<a id="org0d56f34"></a>

 90 Transient


<a id="orgf4cb7b7"></a>

 95 Transient - Not supernova??




# Table of Contents

1.  [Notes from 17 Oct:](#org23fbe57)


<a id="org23fbe57"></a>

# Notes from 17 Oct:

-   try things in log
-   TODO visualize in log -> human readable!
-   (our) preprocessing is good actually not yours
-   Scale time units
-   Measure of goodness given by optimization
-   Merge weirdness? Plot weirdness?
-   Apply ty -> Log -> Apply dy -> Exp



In [ ]:
import numpy as np
import scipy.stats as sps
import scipy.optimize as spo
import matplotlib.pyplot as plt
import corner
import sqlite3

In [ ]:
from justice import simulate as sim
from justice import summarize as summ
from justice import visualize as vis
from justice import xform
from justice import lightcurve
from justice.datasets import plasticc_data

6, 16, 53, 65, 92

In [ ]:
for target_cls in [92]:
    lcs = lightcurve.PlasticcDatasetLC.get_lc_by_target("data/plasticc_training_data.db",target_cls)
    print(str(target_cls))
    for i in range(5):
        curves = lcs[i]
        fig = vis.plot_lcs(curves, title=target_cls, plot_period=True)

In [ ]:
curve1 = lcs[0]
for curve2 in lcs[:25]:
    twocurve = curve2
    result = summ.opt_arclen(curve1, twocurve)
    vis.plot_arclen_res(lcs[0], twocurve, result)
    curve1 = curve1 + result.transform(twocurve)
# onecurve = lcs[0]
# twocurve = lcs[4]

In [ ]:
vis.plot_lcs(curve1)

In [ ]:
result = summ.opt_arclen(onecurve, twocurve)

In [ ]:
vis.plot_arclen_res(onecurve, twocurve, result)

In [ ]:
# import george
# from george import kernels

# x = singlecurve.bands['g'].time[:25]
# y = singlecurve.bands['g'].flux[:25]
# yerr = singlecurve.bands['g'].flux_err[:25]

# kernel = np.mean(yerr) * kernels.Matern32Kernel(500.0, ndim=1)#, axes=0)
# gp = george.GP(kernel)
# gp.compute(x, yerr)

# x_pred = np.linspace(min(x)-10, max(x)+10, 500)
# pred, pred_var = gp.predict(y, x_pred, return_var=True)

# plt.fill_between(x_pred, pred - np.sqrt(pred_var)+100., pred + np.sqrt(pred_var)+100.,
#                 color="k", alpha=0.2)
# plt.plot(x_pred, pred+100., "k", lw=1.5, alpha=0.5)
# plt.errorbar(x, y+100., yerr=yerr, fmt=".k", capsize=0)
# plt.xlabel("x")
# plt.ylabel("y")
# plt.semilogy()